In [1]:
import keras
import keras.layers as KL
import keras.backend as K
from keras.models import Model
import numpy as np

Using TensorFlow backend.


In [2]:
def custom_loss1(y_true, y_pred):
    y_true = K.reshape(y_true,(-1,2))
    return K.mean(K.abs(y_true-y_pred))

def custom_loss2(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred))

In [3]:
input_tensor1 = KL.Input((32,32,3))
input_tensor2 = KL.Input((4,))
input_taget2 = KL.Input((2,))

x = KL.BatchNormalization(axis=-1)(input_tensor1)
x = KL.Conv2D(16,(3,3),padding="same")(x)
x = KL.Activation("relu")(x)
x = KL.MaxPool2D(2)(x)
x = KL.Conv2D(32,(3,3),padding="same")(x)
x = KL.Activation("relu")(x)
x = KL.MaxPool2D(2)(x)
x = KL.Flatten()(x)
x = KL.Dense(32)(x)
x_out2 = KL.Dense(2, name="out2")(x)

y = KL.Dense(32)(input_tensor2)
y = KL.Dense(2, name="out1")(y)


loss1 = KL.Lambda(lambda x: custom_loss1(*x), name="loss1")([y, x_out2])
loss2 = KL.Lambda(lambda x: custom_loss2(*x), name="loss2")([input_taget2, y])

model = Model([input_tensor1, input_tensor2, input_taget2],[x_out2, y, loss1, loss2])

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [11]:
from keras.utils.vis_utils    import plot_model
plot_model(model, to_file="model.png",show_shapes=True)

In [5]:
loss_lay1 = model.get_layer("loss1").output
loss_lay2 = model.get_layer("loss2").output

model.add_loss(loss_lay1)
model.add_loss(loss_lay2)

In [6]:
model.compile(optimizer="sgd", loss=[None,None,None, None])

In [7]:
def data_produce(Num):
    for i in range(Num):
        yield [np.random.normal(1,1,(1,32,32,3)), np.random.normal(1,1,(1,4)), np.random.normal(1,1,(1,2))],[]

In [9]:
dataSet = data_produce(20000)

In [10]:
model.fit_generator(dataSet, steps_per_epoch=1000, epochs=20)

Epoch 1/20
1000/1000 [==============================] - 4s 4ms/step - loss: 1.3024
Epoch 2/20
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0903
Epoch 3/20
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0657
Epoch 4/20
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0651
Epoch 5/20
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0844
Epoch 6/20
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0564
Epoch 7/20
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0692
Epoch 8/20
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0656
Epoch 9/20
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0283
Epoch 10/20
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0520
Epoch 11/20
1000/1000 [==============================] - 3s 3ms/step - loss: 0.9825
Epoch 12/20
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0335
E